# ResNet meets Split-Attention: replication, ablation and analysis of ResNeSt
# Transfer learning subsection

*Authors: Emilie OURAOU, Cyprien QUEMENEUR, Hugo RODET*


<img src="https://pytorch.org/assets/images/resnest.jpg" alt="alt" width="50%"/>

### Abstract
ResNeSt is an architecture of neural networks, applied to computer vision, proposing a mix between a modern convolutive neural network (CNN), the ResNest, as well as the attention mechanism. For our final project we propose a study of the article which introduced the ResNeSts and conduct some additional analyses and ablations. This work indirectly follows our article presentation which focused on vision transformers (ViTs).

ResNeSts can be trained as a general model on image classification then fine-tuned on more downstream tasks (aka transfer learning). This notebook reproduces the experiments of the pretrained models made available by the authors of the article on semantic segmentation, instance segmentation and object detection.

### References

 - [ResNeSt: Split-Attention Networks](https://arxiv.org/abs/2004.08955).

## Semantic segmentation on ADE20k

### A simple example

The links to download the pretrained ResNeSt models for semantic segmentation are available on:

- Gluon (https://cv.gluon.ai/model_zoo/segmentation.html)
- The first author's personal website (https://hangzhang.org/PyTorch-Encoding/model_zoo/segmentation.html)

We first test the pretrained model on a single image.

In [ ]:
import torch
import encoding

In [ ]:
!pip install git+https://github.com/zhanghang1989/PyTorch-Encoding/

  Cloning https://github.com/zhanghang1989/PyTorch-Encoding/ to /tmp/pip-req-build-l4kp7bi2
  Running command git clone -q https://github.com/zhanghang1989/PyTorch-Encoding/ /tmp/pip-req-build-l4kp7bi2
     |████████████████████████████████| 154 kB 5.1 MB/s 
  Created wheel for torch-encoding: filename=torch_encoding-1.2.2b20220417-cp37-cp37m-linux_x86_64.whl size=9153068 sha256=1f64cddd24e8c0ca370aff102c57a3512c148517b86ff0399c2c05ef6243d508
  Stored in directory: /tmp/pip-ephem-wheel-cache-dqwic_uh/wheels/d4/83/00/5a9524a23f206528125bf76110b7304681afa76adbba084b5c
Successfully built torch-encoding


We load the pretrained model ResNeSt50 from the net and load the image.

In [ ]:
model = encoding.models.get_model('DeepLab_ResNeSt50_PContext', pretrained=True).cuda()
model.eval()

url = 'https://github.com/zhanghang1989/image-data/blob/master/' + \
      'encoding/segmentation/pcontext/2010_001829_org.jpg?raw=true'
filename = 'example.jpg'
img = encoding.utils.load_image(
    encoding.utils.download(url, filename)).cuda().unsqueeze(0)

Predicting the result and visualising it. The image is stored on the drive.

In [ ]:
output = model.evaluate(img)
predict = torch.max(output, 1)[1].cpu().numpy() + 1

mask = encoding.utils.get_mask_pallete(predict, 'pascal_voc')
mask.save('output.png')

Model file /root/.encoding/models/resnest50-fb9de5b3.pth is not found. Downloading.


100%|██████████| 107689/107689 [00:07<00:00, 13739.36KB/s]


Model file /root/.encoding/models/deeplab_resnest50_pcontext-08dccbc4.pth is not found. Downloading.


172046KB [00:07, 21675.93KB/s]                            


### Conducting the experiments

We will now replicate the results of the paper for the pretrained models fine-tuned for semantic segmentation. The dataset used is ADE20K. 

The original paper also includes experiments on CityScapes, but contrary to what is announced we could not find the pretrained model for this dataset.

For more information on these two datasets see:
* https://groups.csail.mit.edu/vision/datasets/ADE20K/
* https://www.cityscapes-dataset.com/

We will access the necessary file from the drive and run the experiments with Colab. For this to work you will need to download the PyTorch-Encoding repository and add it to your drive (all rights reserved). The link to the repository is the following: 
* https://github.com/zhanghang1989/PyTorch-Encoding

In [ ]:
from google.colab import drive

Mounted at /content/drive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


You may need to update the path according to your drive's files organisation.

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/PyTorch-Encoding && python setup.py install

-- Building version 1.2.2b20220417
c++:  ['/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/cpu/roi_align_cpu.cpp', '/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/cpu/syncbn_cpu.cpp', '/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/cpu/encoding_cpu.cpp', '/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/cpu/rectify_cpu.cpp', '/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/cpu/operator.cpp', '/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/cpu/nms_cpu.cpp']
cuda:  ['/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/gpu/operator.cpp', '/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/gpu/encoding_kernel.cu', '/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/gpu/lib_ssd.cu', '/content/drive/MyDrive/Colab Notebooks/PyTorch-Encoding/encoding/lib/gpu/roi_align_kernel.cu', '/content/drive/MyDrive/Colab Notebo

Downloading and preparing the dataset ADE20k

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/PyTorch-Encoding && python scripts/prepare_ade20k.py

944710KB [00:12, 72823.05KB/s]                
100% 206856/206856 [00:03<00:00, 67270.39KB/s]


Loading and evaluating ResNeST50 on the validation set.

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/PyTorch-Encoding/experiments/segmentation/ && python test.py --dataset ADE20K --model-zoo DeepLab_ResNeSt50_ADE --eval

Namespace(acc_bn=False, aux=False, backbone='resnet50', base_size=520, batch_size=16, crop_size=480, cuda=True, dataset='ADE20K', eval=True, export=None, model='encnet', model_zoo='DeepLab_ResNeSt50_ADE', no_cuda=False, no_val=False, resume=None, se_loss=False, se_weight=0.2, seed=1, test_batch_size=16, test_folder=None, test_val=False, train_split='train', verify=None, workers=16)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Model file /root/.encoding/models/resnest50-fb9de5b3.pth is not found. Downloading.
100% 107689/107689 [00:08<00:00, 12596.27KB/s]
Model file /root/.encoding/

Evaluation results for ResNeSt50 on ADE20k validation set:
* pixAcc: 0.8117
* mIoU: 0.4512

Loading and evaluating ResNeST101 on the validation set.

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/PyTorch-Encoding/experiments/segmentation/ && python test.py --dataset ADE20K --model-zoo DeepLab_ResNeSt101_ADE --eval

Namespace(acc_bn=False, aux=False, backbone='resnet50', base_size=520, batch_size=16, crop_size=480, cuda=True, dataset='ADE20K', eval=True, export=None, model='encnet', model_zoo='DeepLab_ResNeSt101_ADE', no_cuda=False, no_val=False, resume=None, se_loss=False, se_weight=0.2, seed=1, test_batch_size=16, test_folder=None, test_val=False, train_split='train', verify=None, workers=16)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Model file /root/.encoding/models/resnest101-966fb78c.pth is not found. Downloading.
189242KB [00:09, 19363.69KB/s]                
Model file /root/.encodin

Evaluation results for ResNeSt101 on ADE20k validation set:
* pixAcc: 0.8207
* mIoU: 0.4691

Loading and evaluating ResNeST200 on the validation set.

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/PyTorch-Encoding/experiments/segmentation/ && python test.py --dataset ADE20K --model-zoo DeepLab_ResNeSt200_ADE --eval

Namespace(acc_bn=False, aux=False, backbone='resnet50', base_size=520, batch_size=16, crop_size=480, cuda=True, dataset='ADE20K', eval=True, export=None, model='encnet', model_zoo='DeepLab_ResNeSt200_ADE', no_cuda=False, no_val=False, resume=None, se_loss=False, se_weight=0.2, seed=1, test_batch_size=16, test_folder=None, test_val=False, train_split='train', verify=None, workers=16)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Model file /root/.encoding/models/resnest200-d7fd712f.pth is not found. Downloading.
100% 275389/275389 [00:12<00:00, 21324.48KB/s]
Model file /root/.encodin

Evaluation results for ResNeSt200 on ADE20k validation set:
* pixAcc: 0.8245
* mIoU: 0.4836




The final results match those found in the original paper, the pretrained models thus perform as expected.

model\metrics  | pixAcc | mIoU |
------------------|------------------|------------------|
**ResNeSt50**    | 0.8117 | 0.4512 | 
**ResNeSt101**   | 0.8207 | 0.4691 | 
**ResNeSt200**   | 0.8245 | 0.4836 | 

Please consult the final report for additional comments.

## Instance segmentation

For conducting the experiments for instance segmentation and object detection we will need to load and prepare the dataset COCO. The pretrained models use a Detectron2 wrapper to function so we will also need to install this library.

We will access the necessary file from the drive and run the experiments with Colab. For this to work you will need to download the ResNeSt repository and add it to your drive (all rights reserved).

Link to the ReSNeSt repository:
* https://github.com/zhanghang1989/ResNeSt

Most of the documentation needed here is available in the detectron2 wrapper subsection as well as in the detectron2 repository.  
* https://github.com/zhanghang1989/ResNeSt/tree/master/d2  
* https://github.com/facebookresearch/detectron2/blob/main/GETTING_STARTED.md

In the wrapper subsection you will find both the configuration files as well as the weights, saved under the .pth format. The weights need to be downloaded manually and added to the repository, the path should be the following:
* ResNeSt-master/d2/checkpoints/COCO-InstanceSegmentation/someweights.pth

The configuration files are available by default when cloning the repository, it is not needed to download them.

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


You main need to update the path according to your drive's files organisation.

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master && python setup.py install

-- Building version 0.0.6b20220430
running install
running bdist_egg
running egg_info
writing resnest.egg-info/PKG-INFO
writing dependency_links to resnest.egg-info/dependency_links.txt
writing requirements to resnest.egg-info/requires.txt
writing top-level names to resnest.egg-info/top_level.txt
reading manifest file 'resnest.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'resnest.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying resnest/version.py -> build/lib/resnest
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/resnest
copying build/lib/resnest/__init__.py -> build/bdist.linux-x86_64/egg/resnest
copying build/lib/resnest/utils.py -> build/bdist.linux-x86_64/egg/resnest
creating build/bdist.linux-x86_64/egg/resnest/d2
copying build/lib/resnest/d2/config.py -> build/bdist.linux-x86_64/egg/resnest/d2
copying build/lib/resnest/d2/__init__.py -> build/bdist.l

Installation of detectron2. For an example on how to use detectron2 in colab see:
* https://colab.research.google.com/drive/1uK6zkmNEbdRcSv1gL2GFG8rk24T9IMtZ

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-_q0x50c6
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-_q0x50c6
     |████████████████████████████████| 74 kB 2.8 MB/s 
     |████████████████████████████████| 147 kB 17.2 MB/s 
     |████████████████████████████████| 130 kB 67.1 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 749 kB 62.0 MB/s 
     |████████████████████████████████| 843 kB 61.5 MB/s 
     |████████████████████████████████| 596 kB 57.0 MB/s 
     |████████████████████████████████| 112 kB 73.7 MB/s 
  Created wheel for detectron2: filename=detectron2-0.6-cp37-cp37m-linux_x86_64.whl size=5286162 sha256=26defbfbcd1682b97fc83f3ab10cbfdf09636db8a5704b211be65dfdeb1dbe4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-0z3dmo1q/wheels/07/dc/32/0322cb484dbefab8b9366bfedbaff5060ac7d149d69c27ca5d
  Created wheel for antlr4-python3-runtim

Downloading and preparing the dataset COCO. For more information on this dataset see: https://cocodataset.org/#home.

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2/datasets && python prepare_coco.py

18883655KB [07:23, 42600.08KB/s]                  
796471KB [00:21, 37923.51KB/s]                
246981KB [00:06, 39193.58KB/s]                
100% 432/432 [00:00<00:00, 4898.18KB/s]
19KB [00:00, 992.67KB/s]  
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (576/576), done.
running install
running bdist_egg
running egg_info
creating pycocotools.egg-info
writing pycocotools.egg-info/PKG-INFO
writing dependency_links to pycocotools.egg-info/dependency_links.txt
writing requirements to pycocotools.egg-info/requires.txt
writing top-level names to pycocotools.egg-info/top_level.txt
writing manifest file 'pycocotools.egg-info/SOURCES.txt'
writing manifest file 'pycocotools.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib.l

cd into the dataset. We need a training set, a validation set and the annotations set to perform the experiments.

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2/datasets/coco && ls

annotations		      train2017      val2017
annotations_trainval2017.zip  train2017.zip  val2017.zip


In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_50_FPN_syncBN_1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_50_FPN_syncBN_1x-c58bd325.pth

Command Line Args: Namespace(config_file='./configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_50_FPN_syncBN_1x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', './checkpoints/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_50_FPN_syncBN_1x-c58bd325.pth'], resume=False)
[04/30 13:31:58 detectron2]: Rank of current process: 0. World size: 1
[04/30 13:31:59 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
numpy                   1.21.6
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.5
CUDA compiler           CUDA 11.1
detectron2 arch flags   7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.11.0+cu113 @/usr/local/lib/python3.7/dist-packages/torch
PyTorch

mask_cascade_rcnn_ResNeSt_50 - evaluation results for coco_2017_val in csv format:

Task\metrics  | AP | AP50 | AP75 | APs | APm | APl |
--------------|----|------|------|-----|-----|-----|
**bbox**    | 46.2371 | 64.6233 | 50.1493 | 28.8566 | 49.8315 | 58.9055 
**segm**   | 39.6405 | 61.8638 | 42.8804 | 20.8359 | 42.4238 | 55.5196 


In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_101_FPN_syncBN_1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_101_FPN_syncBN_1x-62448b9c.pth

Command Line Args: Namespace(config_file='./configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_101_FPN_syncBN_1x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', './checkpoints/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_101_FPN_syncBN_1x-62448b9c.pth'], resume=False)
[04/30 14:34:35 detectron2]: Rank of current process: 0. World size: 1
[04/30 14:34:35 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
numpy                   1.21.6
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.5
CUDA compiler           CUDA 11.1
detectron2 arch flags   7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.11.0+cu113 @/usr/local/lib/python3.7/dist-packages/torch
PyTor

mask_cascade_rcnn_ResNeSt_101 - evaluation results for coco_2017_val in csv format:

Task\metrics  | AP | AP50 | AP75 | APs | APm | APl |
--------------|----|------|------|-----|-----|-----|
**bbox**    | 46.2371 | 64.6233 | 50.1493 | 28.8566 | 49.8315 | 58.9055 
**segm**   | 39.6405 | 61.8638 | 42.8804 | 20.8359 | 42.4238 | 55.5196 

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-InstanceSegmentation/mask_rcnn_ResNeSt_50_FPN_syncBN_1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/mask_rcnn_ResNeSt_50_FPN_syncBN_1x-f442d863.pth

Command Line Args: Namespace(config_file='./configs/COCO-InstanceSegmentation/mask_rcnn_ResNeSt_50_FPN_syncBN_1x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', './checkpoints/mask_rcnn_ResNeSt_50_FPN_syncBN_1x-f442d863.pth'], resume=False)
[04/30 14:41:08 detectron2]: Rank of current process: 0. World size: 1
[04/30 14:41:09 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
numpy                   1.21.6
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.5
CUDA compiler           CUDA 11.1
detectron2 arch flags   7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.11.0+cu113 @/usr/local/lib/python3.7/dist-packages/torch
PyTorch debug build     False
GPU available      

mask_rcnn_ResNeSt_50 - evaluation results for coco_2017_val in csv format:

Task\metrics  | AP | AP50 | AP75 | APs | APm | APl |
--------------|----|------|------|-----|-----|-----|
**bbox**    | 46.2371 | 64.6233 | 50.1493 | 28.8566 | 49.8315 | 58.9055 
**segm**   | 39.6405 | 61.8638 | 42.8804 | 20.8359 | 42.4238 | 55.5196 

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-InstanceSegmentation/mask_rcnn_ResNeSt_101_FPN_syncBN_1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/mask_rcnn_ResNeSt_101_FPN_syncBN_1x-528502c6.pth

mask_rcnn_ResNeSt_101 - evaluation results for coco_2017_val in csv format:

Task\metrics  | AP | AP50 | AP75 | APs | APm | APl |
--------------|----|------|------|-----|-----|-----|
**bbox**    | 46.2371 | 64.6233 | 50.1493 | 28.8566 | 49.8315 | 58.9055 
**segm**   | 39.6405 | 61.8638 | 42.8804 | 20.8359 | 42.4238 | 55.5196 

Overral, the final results match those found in the original paper, the pretrained models thus perform as expected.

Method | Backbone  | box mAP% | mask mAP% |
------------------|--------------|------------------|------------------|
**Mask-RCNN**    | ResNeSt50 | 42.81 |  38.14
**Mask-RCNN**   | ResNeSt101 | 45.75 |  40.65
**Cascade-RCNN**   | ResNeSt50 | 46.19 |  39.55
**Cascade-RCNN** | ResNeSt101 | 48.30 | 41.56 

Please consult the final report for additional comments.

## Object Detection

The object detection task closely ressembles the instance segmentation task, in both cases the dataset evaluated on is COCO and the models are based on a Detectron2 wrapper

Like before we will access the necessary file from the drive and run the experiments with Colab and the ResNeSt repository has to be loaded in the drive. We assume that the dataset has already been prepared in the above section and the repository and detectron2 were successfully installed.

In the wrapper subsection you will also find both the configuration files as and the weights, for this experiment. Once again the weights need to be downloaded manually and added to the repository, the path should be the following:
* ResNeSt-master/d2/checkpoints/COCO-ObjectDetection/someweights.pth

The configuration files are available by default when cloning the repository, it is not needed to download them.

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2/checkpoints/COCO-ObjectDetection/ && ls

faster_cascade_rcnn_ResNeSt_50_FPN_syncbn_range-scale-1x-e9955232.pth


In [8]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-Detection/faster_cascade_rcnn_ResNeSt_50_FPN_syncbn_range-scale-1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/COCO-ObjectDetection/faster_cascade_rcnn_ResNeSt_50_FPN_syncbn_range-scale-1x-e9955232.pth

Command Line Args: Namespace(config_file='./configs/COCO-Detection/faster_cascade_rcnn_ResNeSt_50_FPN_syncbn_range-scale-1x.yaml', dist_url='tcp://127.0.0.1:49152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=['MODEL.WEIGHTS', './checkpoints/COCO-ObjectDetection/faster_cascade_rcnn_ResNeSt_50_FPN_syncbn_range-scale-1x-e9955232.pth'], resume=False)
[04/30 15:22:38 detectron2]: Rank of current process: 0. World size: 1
[04/30 15:22:38 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.13 (default, Apr 24 2022, 01:04:09) [GCC 7.5.0]
numpy                   1.21.6
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.5
CUDA compiler           CUDA 11.1
detectron2 arch flags   7.5
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.11.0+cu113 @/usr/local/lib/python3.7/dist-packages/t

[04/30 15:56:11 d2.engine.defaults]: Evaluation results for coco_2017_val in csv format:
[04/30 15:56:11 d2.evaluation.testing]: copypaste: Task: bbox
[04/30 15:56:11 d2.evaluation.testing]: copypaste: AP,AP50,AP75,APs,APm,APl
[04/30 15:56:11 d2.evaluation.testing]: copypaste: 45.4079,63.9231,48.7010,28.7674,48.6927,58.4289

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-Detection/faster_cascade_rcnn_ResNeSt_101_FPN_syncbn_range-scale_1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/COCO-ObjectDetection/faster_cascade_rcnn_ResNeSt_101_FPN_syncbn_range-scale_1x-3627ef78.pth

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-Detection/faster_cascade_rcnn_ResNeSt_200_FPN_syncbn_range-scale_1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/COCO-ObjectDetection/faster_cascade_rcnn_ResNeSt_200_FPN_syncbn_range-scale_1x-1be2a87e.pth

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-Detection/faster_rcnn_ResNeSt_50_FPN_syncbn_range-scale_1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/COCO-ObjectDetection/faster_rcnn_ResNeSt_50_FPN_syncbn_range-scale_1x-ad123c0b.pth

In [ ]:
!cd drive/MyDrive/'Colab Notebooks'/ResNeSt-master/d2 && python train_net.py  \
 --config-file ./configs/COCO-Detection/faster_rcnn_ResNeSt_101_FPN_syncbn_range-scale_1x.yaml \
 --eval-only MODEL.WEIGHTS ./checkpoints/COCO-ObjectDetection/faster_rcnn_ResNeSt_101_FPN_syncbn_range-scale_1x-d8f284b6.pth

# Conclusion

We were ultimately able to reproduce all the experiments conducted on the pretrained models for the transfer learning tasks. ResNeSts indeed constitute an improvement to ResNets as backbone models for downstream tasks, at the same computational cost.

The replications were conducted on semantic segmentation, instance segmentation and objet detection, with different level of ease. Semantic segmentation was easy to reproduce on ADE20k but the pretrained models are missing for CityScapes on both the GluonCV platform and the original article first author's website. Furthermore, we encountered several difficulties on instance segmentation and object detection because the documentation was not as clear, in particular concerning the match between the configurations files and the weight files. The task would have been made easier if it was possible to download the pretrained models from the net with pip, like in the other cases. Yet, we were still able to reproduce the experiments successfully.

We chose to conduct the experiments on Colab. All things considered, opting immediatly for an AWS instance could have simplified the ordeal, for we have experienced many memory errors using Colab Pro.

For further comments and analyses please consult our final report.